<a href="https://colab.research.google.com/github/mcadete/LLM_4_Biz_16/blob/main/Tech16_class3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%pip install llama-index

In [22]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)


In [23]:
%pip install pypdf

In [24]:
!wget https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf

--2025-02-13 04:07:59--  https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf
Resolving privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)... 184.26.127.80, 184.26.127.73
Connecting to privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)|184.26.127.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘2025-isg-outlook.pdf.1’

2025-isg-outlook.pd     [ <=>                ]   6.88M  --.-KB/s    in 0.1s    

2025-02-13 04:07:59 (62.9 MB/s) - ‘2025-isg-outlook.pdf.1’ saved [7209986]



In [25]:
import os

os.environ["OPENAI_API_KEY"] = open_ai_key

In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


load the documents into the vector store

In [27]:
documents = SimpleDirectoryReader('./').load_data('2025-isg-outlook.pdf')

Loading files: 100%|██████████| 2/2 [00:08<00:00,  4.22s/file]


In [28]:
documents[10]

Document(id_='de64ad95-ba1d-4223-818b-bbf3d2102c8a', embedding=None, metadata={'page_label': '9', 'file_name': '2025-isg-outlook.pdf', 'file_path': '/content/2025-isg-outlook.pdf', 'file_type': 'application/pdf', 'file_size': 7209986, 'creation_date': '2025-02-13', 'last_modified_date': '2025-02-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='9Outlook Investment Strategy Group\n SECTION II: DIVERGING PATHS\n58  2025 Global  \nEconomic Outlook\n  Despite divergent paths, most economies will \nascend toward higher elevations this year.  \n60  United States\n64  Eurozone\n66  United Kingdom\n67 Japan\n68  Emerging Markets\n SECTION III:

creating a vector embeddings representations for the document. reading it, tokenizing it, giving it numerical representation

In [29]:
index = VectorStoreIndex.from_documents(documents)

In [30]:
query_engine = index.as_query_engine()

In [42]:
# response = query_engine.query("What was the 2024 US GDP in percentage terms?")
response = query_engine.query("What is the outlook for 2025 for US GDP? what was 2024 US GDP in percentage terms compared to 2023")
print(response)

The outlook for 2025 for US GDP is an estimated growth of 2.3%. In 2024, US GDP grew by 2.8%, which was higher compared to the growth rate in 2023.


In [38]:
print(response)

The 2024 US GDP increased by $1.4 trillion, which was higher than the GDP growth in the Eurozone and China.


Adding memory. the model does not have memory. effectively we need to build a chat bot so that the system has memory for reasoning

In [44]:
response = query_engine.query("What did I just ask you? Give me the exact question")
print(response)

What did I just ask you? Give me the exact question.


In [50]:
from re import VERBOSE
chat_engine = index.as_chat_engine(chat_mode="openai", verbose=True)
response = chat_engine.chat("hello there")
print(response)

Added user message to memory: hello there
Hello! How can I assist you today?


In [51]:
response = chat_engine.chat("What is the liklihood of a recession in 2025?")
print(response)

Added user message to memory: What is the liklihood of a recession in 2025?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What is the likelihood of a recession in 2025?"}
Got output: The likelihood of a recession in 2025 is 20%, which is close to the long-term average but below the current consensus.

The likelihood of a recession in 2025 is estimated to be around 20%, which is close to the long-term average but below the current consensus.


In [52]:
response = chat_engine.chat("What investement is projected to give the largest returns in 2025?")
print(response)

Added user message to memory: What investement is projected to give the largest returns in 2025?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What investment is projected to give the largest returns in 2025?"}
Got output: Aerospace and defense sector is projected to give the largest returns in 2025.

The aerospace and defense sector is projected to give the largest returns in 2025.


In [53]:
response = chat_engine.chat("What did I just ask you?")
print(response)

Added user message to memory: What did I just ask you?
You asked, "What investment is projected to give the largest returns in 2025?"


In [54]:
response = chat_engine.chat("What is the outlook for S&P500 in 2025?")
print(response)

Added user message to memory: What is the outlook for S&P500 in 2025?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What is the outlook for S&P500 in 2025?"}
Got output: The outlook for the S&P 500 in 2025 suggests high-single-digit total returns with an implied target range of 6,200–6,300. Earnings growth is expected to be a key driver of S&P 500 returns, with a favorable skew towards upside surprises. The market may experience some compression in the P/E ratio, leading to high-single-digit total returns. Additionally, the S&P 500 borrowing costs are anticipated to be relatively immune to higher interest rates over the next several years, which could further support the market performance.

The outlook for the S&P 500 in 2025 suggests high-single-digit total returns with an implied target range of 6,200–6,300. Earnings growth is expected to be a key driver of S&P 500 returns, with a favorable skew towards upside surprises. The market may experience 

changing settings - chunking (llama_index is ~ 1000 chars)

In [62]:
from llama_index.core import Settings

Settings.chunk_size = 20
Settings.chunk_overlap = 2


In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What ETF has the best outlook, highest returns, in 2025?")
print(response)

Metadata length (18) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (18) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (19) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (19) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (19) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (19) is close to chunk size (20). Resulting chunks are less than 50 tokens

Adding a Vector DB - ChromaDB

In [ ]:
%pip install llama-index-vector-stores-chroma

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

instantiated the db

In [ ]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

created the table

In [ ]:
chroma_collection = chroma_client.create_collection("tech16")

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('./').load_data('2025-isg-outlook.pdf')
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

query_engine = index.as_query_engine()
response = query_engine.query("What ETF has the best outlook, highest returns, in 2025?")
print(response)